In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
K = keras.backend

from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Загрузим эмбеддинги

In [2]:
class Vocab(object):
    def __init__(self):
        self.rev_vocabulary_mapping = []
        self.vocabulary_mapping = {}
        self.embeddings = None
    
    def unvectorize(self, ids):
        return ' '.join([self.rev_vocabulary_mapping[i] for i in ids])
    
    def vectorize(self, tokens):
        return [self.vocabulary_mapping.get(w, self.vocabulary_mapping['<UNK>']) for w in tokens]
    
    def load_embeddings(self, filename, embedding_size=50, cut_at=400000):
        # список векторов
        embeddings_list = []
        # словарь слово-индекс
        self.vocabulary_mapping = {'<PAD>': 0} # занесём ключ соотетствующий отступу
        self.rev_vocabulary_mapping.append('<PAD>')
        pad = np.zeros(embedding_size) # создадим вектор для PAD
        embeddings_list.append(pad)

        with open(filename,encoding='utf8') as glove_file: 
            i = 1
            for line in glove_file:
                fields = line.split()
                token = fields[0]
                vector = np.array(list(map(float, fields[1:])))
                
                embeddings_list.append(vector)
                self.vocabulary_mapping[token] = i
                self.rev_vocabulary_mapping.append(token)
                i += 1
                if i >= cut_at:
                    break
                    
        self.vocabulary_mapping['<UNK>'] = len(embeddings_list)
        self.rev_vocabulary_mapping.append('<UNK>')
        unk = np.mean(embeddings_list[1:], axis=0) # считаем средний вектор
        embeddings_list.append(unk)

        self.embeddings = np.array(embeddings_list)
                
        print(f'Loaded {self.embeddings.shape} embeddings')        

In [3]:
vocab = Vocab()

In [4]:
vocab.load_embeddings('glove/glove.6B.50d.txt')

Loaded (400001, 50) embeddings


## Загрузим ATIS

In [5]:
def load_csv(filename):
    results = []
    with open(filename) as f:
        for l in f:
            results.append(l.strip())
            
    return results

Загрузим "словари" сущностей, классов и вокабуляр

In [6]:
slots = load_csv('data/atis.dict.slots.csv')
intents = load_csv('data/atis.dict.intent.csv')
atis_vocab = load_csv('data/atis.dict.vocab.csv')

Загрузим тренировочные данные

In [7]:
atis_train_queries = load_csv('data/atis.train.query.csv')
atis_train_intents = load_csv('data/atis.train.intent.csv')
atis_train_slots = load_csv('data/atis.train.slots.csv')

In [8]:
atis_train_queries = [[int(index) for index in line.split()] for line in atis_train_queries]
atis_train_intents = [int(index) for index in atis_train_intents]
atis_train_slots = [[int(index) for index in line.split()] for line in atis_train_slots]

Взглянем на данные

In [9]:
[' '.join([atis_vocab[i] for i in line]) for line in atis_train_queries]

['BOS i want to fly from boston at 838 am and arrive in denver at 1110 in the morning EOS',
 'BOS what flights are available from pittsburgh to baltimore on thursday morning EOS',
 'BOS what is the arrival time in san francisco for the 755 am flight leaving washington EOS',
 'BOS cheapest airfare from tacoma to orlando EOS',
 'BOS round trip fares from pittsburgh to philadelphia under 1000 dollars EOS',
 'BOS i need a flight tomorrow from columbus to minneapolis EOS',
 'BOS what kind of aircraft is used on a flight from cleveland to dallas EOS',
 'BOS show me the flights from pittsburgh to los angeles on thursday EOS',
 'BOS all flights from boston to washington EOS',
 'BOS what kind of ground transportation is available in denver EOS',
 'BOS show me the flights from dallas to san francisco EOS',
 'BOS show me the flights from san diego to newark by way of houston EOS',
 "BOS what 's the airport at orlando EOS",
 'BOS what is the cheapest flight from boston to bwi EOS',
 'BOS all fligh

In [10]:
atis_train_queries_w = [[atis_vocab[i] for i in line] for line in atis_train_queries]

In [11]:
train_sent = [vocab.vectorize(tokens[1:-1]) for tokens in atis_train_queries_w]
train_slots = [i[1:-1] for i in atis_train_slots]

In [40]:
train_labels = atis_train_intents

Не забудьте, что есть ещё и тестовый набор данных. На нём нужно будет посчитать итоговую точность для каждой из задач

## Собираем модель для классификации

In [555]:
def build_classification_model(embeddings):
    # Точка входа в граф задаётся при помощи специальных тензоров типа Input
    # Первая координата соответсвует длине текста, так как тексты в датасете имеют разную длину
    # значение считается переменным
    text_input = keras.layers.Input(shape=(None,), dtype='int32')
    
    # Создаём специальный слой для работы с embedding, 
    # Его функция -- заменять индентификатор вектором из Glove
    # Указываем trainable = False, чтобы векторы embedding'ов не изменялись в процессе обучения
    embedding_layer = keras.layers.Embedding(input_dim = embeddings.shape[0], 
                                       output_dim = embeddings.shape[1], 
                                       weights=[embeddings],
                                       trainable = False)
    
    x = embedding_layer(text_input)
    
    # Создаём рекуррентную ячейку
    # Первый параметр отвечает за размер внутреннего состояния (памяти ячейки)
    # По умолчанию такой слой возвращает только последнее состояние (см. картинку),
    # Если мы хотим получить состояния на каждом шаге необходимо указать return_sequences = True
    x = keras.layers.LSTM(128, recurrent_dropout=0.25)(x)

    # Полученный результат направляем в полносвязный слой, который будет осуществлять классификацию
    output = keras.layers.Dense(26, activation='softmax')(x)
    
    model = keras.models.Model(inputs=[text_input], outputs=[output], name = 'LSTM_classifier')
    
    return model

In [556]:
keras.backend.clear_session()
model = build_classification_model(vocab.embeddings)

In [557]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 26)                3354      
Total params: 20,095,052
Trainable params: 95,002
Non-trainable params: 20,000,050
_________________________________________________________________


In [558]:
#  Для оптимизации будем использовать Adam 
adam = keras.optimizers.Adam(lr=0.0001)

#Перед испльзованием модель необходимо скомпилировать
model.compile(adam, keras.losses.sparse_categorical_crossentropy, metrics=['acc'])

Разобьём датасет на три части

In [559]:
x_train, x_val, y_train, y_val = train_test_split(train_sent, train_labels, test_size=0.1)

Почти всё готово, чтобы начать обучение. Но так, как все предлдожения разной длины мы не можем конвертировать x в тензор, нам необходимо выровнять длину. Для этого мы воспользуемся специальной функцией pad_sequences(), доступной в keras. Недостающие элементы будут заполнены специальным символом PAD

Но нам не хотелось бы увеличивать все элементы в датасете по размеру максимального. По этогому будем генерировать батчи сами и применять pad_sequences к ним независимо

In [560]:
def generate_batches(x, y, batch_size=64):
    i = 0
    while True:
        i = i % len(x)
        yield keras.preprocessing.sequence.pad_sequences(x[i:i+batch_size]), y[i:i+batch_size]
        i += batch_size

In [561]:
train_generator = generate_batches(x_train, y_train)
val_generator = generate_batches(x_val, y_val)



Теперь обучим нашу модель

In [562]:
cbs = [
    keras.callbacks.ModelCheckpoint('models/LSTM/{epoch:02d}-{val_loss:.4f}.h5', save_best_only=True),
    keras.callbacks.TensorBoard(log_dir='models/LSTM/'),
    keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3, verbose=1, min_lr=0.00001)
]


model.fit_generator(generator= train_generator, 
                    validation_data = val_generator, 
                    validation_steps= 64,
                    steps_per_epoch=256,
                    epochs=5,
                    callbacks=cbs)

Epoch 1/5
256/256 [==============================] - 13s 51ms/step - loss: 1.4110 - acc: 0.7182 - val_loss: 0.9612 - val_acc: 0.7323
Epoch 2/5
256/256 [==============================] - 13s 49ms/step - loss: 0.8441 - acc: 0.7839 - val_loss: 0.6165 - val_acc: 0.8742
Epoch 3/5
256/256 [==============================] - 12s 48ms/step - loss: 0.6392 - acc: 0.8575 - val_loss: 0.5029 - val_acc: 0.8922
Epoch 4/5
256/256 [==============================] - 12s 48ms/step - loss: 0.5381 - acc: 0.8748 - val_loss: 0.4255 - val_acc: 0.8992
Epoch 5/5
256/256 [==============================] - 12s 48ms/step - loss: 0.4587 - acc: 0.8873 - val_loss: 0.3672 - val_acc: 0.9057


## Самостоятельная работа

Необходимо построить и обучить модель, которая осуществляет разметку слов по "сущностям" (slots). 

Обратите внимание, что для этого придётся написать свою функцию потерь. Сигнатура loss_function(y_true, y_pred).

Дополнительно стоит подумать над тем, как правильно считать точность.

После того, как NER модель будет обучена, постройте третью модель, которая решает обе задачи одновременно.

In [563]:
K = keras.backend

def build_classification_model(embeddings):

    text_input = keras.layers.Input(shape=(None,), dtype='int32')
    embedding_layer = keras.layers.Embedding(input_dim = embeddings.shape[0], 
                                       output_dim = embeddings.shape[1], 
                                       weights=[embeddings],
                                       trainable = False)
    
    x = embedding_layer(text_input)
    x = keras.layers.Bidirectional(keras.layers.LSTM(150, recurrent_dropout=0.25, return_sequences=True))(x)
    

    # Полученный результат направляем в полносвязный слой, который будет осуществлять классификацию
    output_slots = keras.layers.Dense(130, activation='softmax')(x)
    model = keras.models.Model(inputs=[text_input], outputs=[output_slots], name = 'LSTM_classifier')
    
    return model

def my_slots_loss(y_true, y_pred, batch_size = 64):

    lst_iter = K.arange(start = 0, stop =K.shape(y_true)[0])
    ke = K.sparse_categorical_crossentropy
    batch_losses = keras.backend.map_fn(lambda i: ke(y_true[i],y_pred[i]),
                                     lst_iter, name=None, dtype='float32')
    losses = K.mean(batch_losses,axis=0)
    
    return losses



In [564]:
keras.backend.clear_session()
model = build_classification_model(vocab.embeddings)

adam = keras.optimizers.Adam(lr=0.0001)
model.compile(adam,loss = my_slots_loss, metrics=['acc'])


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          20000050  
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 300)         241200    
_________________________________________________________________
dense_1 (Dense)              (None, None, 130)         39130     
Total params: 20,280,380
Trainable params: 280,330
Non-trainable params: 20,000,050
_________________________________________________________________


In [565]:
def generate_batches(x, y, batch_size=64):
    i = 0
    while True:
        i = i % len(x)
        yield keras.preprocessing.sequence.pad_sequences(x[i:i+batch_size]), \
              np.expand_dims(keras.preprocessing.sequence.pad_sequences(y[i:i+batch_size],value=129),axis=-1)
        i += batch_size
        
        
x_train, x_val, y_train, y_val = train_test_split(train_sent, train_slots, test_size=0.1)

train_generator = generate_batches(x_train, y_train)
val_generator = generate_batches(x_val, y_val)

steps_per_epoch = int(len(x_train)/64)
steps_per_valid = int(len(x_val)/64)


In [566]:
cbs = [
    keras.callbacks.ModelCheckpoint('models/LSTM/{epoch:02d}-{val_loss:.4f}.h5', save_best_only=True),
    keras.callbacks.TensorBoard(log_dir='models/LSTM/'),
    keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3, verbose=1, min_lr=0.00001)
]


model.fit_generator(generator= train_generator, 
                    validation_data = val_generator, 
                    validation_steps= 64, #steps_per_epoch,
                    steps_per_epoch= 256, #steps_per_valid,
                    epochs=5)

Epoch 1/5
256/256 [==============================] - 32s 123ms/step - loss: 1.8218 - acc: 1.5488e-05 - val_loss: 0.7841 - val_acc: 0.0000e+00
Epoch 2/5
256/256 [==============================] - 30s 116ms/step - loss: 0.6680 - acc: 0.0000e+00 - val_loss: 0.5795 - val_acc: 0.0000e+00
Epoch 3/5
 82/256 [========>.....................] - ETA: 18s - loss: 0.5438 - acc: 0.0000e+00

KeyboardInterrupt: 

In [31]:
## check predicitons
##

x,y  = next(val_generator)
tt = model.predict(x)

#tt = model.predict_generator(val_generator, steps=1)
print (tt.shape)
predictions = [[np.argmax(i) for i in x] for x in tt]
predictions = np.array(predictions)
print (np.unique(predictions))
predictions

(64, 23, 130)
[  1   2  14  21  26  27  28  33  35  36  39  42  44  48  78  83  92  95
 100 110 119 125 128 129]


array([[129, 129, 129, ..., 128, 128,  48],
       [129, 129, 129, ..., 128,  78, 125],
       [129, 129, 129, ..., 128,   2, 128],
       ...,
       [129, 129, 129, ...,  48, 128,  78],
       [129, 129, 129, ..., 128,  78, 128],
       [129, 129, 129, ...,  78, 128,  26]], dtype=int64)

### Combine two models

In [663]:
K = keras.backend

def my_model_two(embeddings):
    # Точка входа в граф задаётся при помощи специальных тензоров типа Input
    # Первая координата соответсвует длине текста, так как тексты в датасете имеют разную длину
    # значение считается переменным
    text_input = keras.layers.Input(shape=(None,), dtype='int32')
    

    embedding_layer = keras.layers.Embedding(input_dim = embeddings.shape[0], 
                                       output_dim = embeddings.shape[1], 
                                       weights=[embeddings],
                                       trainable = False)
    
    x = embedding_layer(text_input)
    x = keras.layers.Bidirectional(keras.layers.GRU(100, recurrent_dropout=0.25, return_sequences=True))(x)

    out1 = keras.layers.Dense(130, activation='softmax', name='output_slots')(x)
    
    x = keras.layers.GRU(100, recurrent_dropout=0.25)(x)

    out2 = keras.layers.Dense(26, activation='softmax', name='output_intents')(x)
    
    model = keras.models.Model(inputs=[text_input], outputs=[out1,out2], name = 'LSTM_classifier')
    
    return model

def my_slots_loss(y_true, y_pred, batch_size = 64):

    lst_iter = K.arange(start = 0, stop =K.shape(y_true)[0])
    ke = K.sparse_categorical_crossentropy
    batch_losses = keras.backend.map_fn(lambda i: ke(y_true[i],y_pred[i]),
                                     lst_iter, name=None, dtype='float32')
    losses = K.mean(batch_losses,axis=0)
    
    return losses


## not finished
def my_accuracy(y_true, y_pred):
    
    class_true = y_true
    class_pred = K.cast(K.argmax(y_pred, axis=-1),dtype='float32')
    
    uval = tf.unique(K.flatten(class_true))[0]
    
    def my_acc(class_true, class_pred, i_cl):
        
        c1 = K.flatten(K.equal(class_true,i_cl))
        c2 = K.flatten(K.equal(class_pred,i_cl))
        true_pos = K.sum(K.cast(K.equal(c1,c2),'float32'))
        tot_label = K.sum(K.cast(c1,'float32'))
        acc = (true_pos+K.epsilon())/(tot_label+K.epsilon())
        return acc
        
    acc = keras.backend.map_fn(lambda i: my_acc(class_true, class_pred, i),uval,dtype='float32') 
    return K.mean(acc)

In [664]:
keras.backend.clear_session()
model = my_model_two(vocab.embeddings)

adam = keras.optimizers.Adam(lr=0.0001)

dict_losses = {'output_slots':my_slots_loss, 'output_intents':keras.losses.sparse_categorical_crossentropy}
dict_weights = {'output_slots': 1., 'output_intents':1.}


# dict_losses = {'output_intents':keras.losses.sparse_categorical_crossentropy}
# dict_weights = {'output_intents':1.}
# dict_losses = {'output_slots':my_slots_loss}
# dict_weights = {'output_slots': 1.}

model.compile(adam, loss = dict_losses, 
              loss_weights=dict_weights,
              metrics=['acc',my_accuracy])


model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     20000050    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, None, 200)    90600       embedding_1[0][0]                
__________________________________________________________________________________________________
gru_2 (GRU)                     (None, 100)          90300       bidirectional_1[0][0]            
__________________________________________________________________________________________________
output_slo

In [665]:
def generate_batches(x, y, y_label, batch_size=64):
    i = 0
    while True:
        i = i % len(x)
        yield keras.preprocessing.sequence.pad_sequences(x[i:i+batch_size]), \
            [np.expand_dims(keras.preprocessing.sequence.pad_sequences(y[i:i+batch_size],value=129),axis=-1),
               np.expand_dims(y_label[i:i+batch_size],axis=-1)]
        i += batch_size
        
        
x_train, x_val, y_train, y_val, y_lab_train, y_lab_val = train_test_split(train_sent, train_slots,train_labels, test_size=0.1)

train_generator = generate_batches(x_train, y_train, y_lab_train)
val_generator = generate_batches(x_val, y_val, y_lab_val)

steps_per_epoch = int(len(x_train)/64)
steps_per_valid = int(len(x_val)/64)

In [666]:
cbs = [
    keras.callbacks.ModelCheckpoint('models/LSTM/{epoch:02d}-{val_loss:.4f}.h5', save_best_only=True),
    keras.callbacks.TensorBoard(log_dir='models/LSTM/'),
    keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3, verbose=1, min_lr=0.00001)
]


model.fit_generator(generator= train_generator, 
                    validation_data = val_generator, 
                    validation_steps= 64, #steps_per_epoch,
                    steps_per_epoch= 256, #steps_per_valid,
                    epochs=5)

Epoch 1/5
256/256 [==============================] - 40s 157ms/step - loss: 3.8342 - output_slots_loss: 2.4984 - output_intents_loss: 1.3358 - output_slots_acc: 0.0000e+00 - output_slots_my_accuracy: 914.4746 - output_intents_acc: 0.7104 - output_intents_my_accuracy: 34.7530 - val_loss: 1.8298 - val_output_slots_loss: 0.8787 - val_output_intents_loss: 0.9511 - val_output_slots_acc: 0.0000e+00 - val_output_slots_my_accuracy: 834.1341 - val_output_intents_acc: 0.7643 - val_output_intents_my_accuracy: 32.8419
Epoch 2/5
256/256 [==============================] - 38s 149ms/step - loss: 1.6133 - output_slots_loss: 0.7183 - output_intents_loss: 0.8950 - output_slots_acc: 0.0000e+00 - output_slots_my_accuracy: 909.8668 - output_intents_acc: 0.7781 - output_intents_my_accuracy: 34.7321 - val_loss: 1.3982 - val_output_slots_loss: 0.6396 - val_output_intents_loss: 0.7586 - val_output_slots_acc: 0.0000e+00 - val_output_slots_my_accuracy: 839.7794 - val_output_intents_acc: 0.8051 - val_output_inten

KeyboardInterrupt: 

### Predict test data

In [480]:
atis_test_queries = load_csv('data/atis.test.query.csv')
atis_test_intents = load_csv('data/atis.test.intent.csv')
atis_test_slots = load_csv('data/atis.test.slots.csv')

atis_test_queries = [[int(index) for index in line.split()] for line in atis_test_queries]
atis_test_intents = [int(index) for index in atis_test_intents]
atis_test_slots = [[int(index) for index in line.split()] for line in atis_test_slots]

atis_test_queries_w = [[atis_vocab[i] for i in line] for line in atis_test_queries]

In [481]:
test_sent = [vocab.vectorize(tokens[1:-1]) for tokens in atis_test_queries_w]
test_slots = [i[1:-1] for i in atis_test_slots]
test_labels = atis_test_intents

In [513]:
batch_size = 64
cnt_batches = int(len(test_sent)/batch_size)

test_pred_slots = []
test_pred_intent = []

for i in range(cnt_batches+1):
    i = i * batch_size
    x = keras.preprocessing.sequence.pad_sequences(test_sent[i:i+batch_size])
    y1, y2 = model.predict(x)
    tmp1 = [[np.argmax(i) for i in x] for x in y1]
    tmp2 = [np.argmax(x) for x in y2]
    
    test_pred_slots += tmp1
    test_pred_intent += tmp2


In [517]:
print ("Final accuracy:", accuracy_score(test_labels,test_pred_intent))

Final accuracy: 0.8365061590145577
